In [ ]:
import logging
import traceback
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
import datetime

# Configuración del logger
logging.basicConfig(
    filename=f"/home/m/Escritorio/RAG/logs/{str(datetime.datetime.today().strftime('%Y-%m-%d'))}_query.log",  # Guardar logs en un archivo
    level=logging.DEBUG,  # Puedes cambiarlo a INFO en producción
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

logging.debug("🧪 Query inicializada correctamente.")


# Ruta al directorio donde está la base vectorial
CHROMA_PATH = "db"

# Cargar embeddings
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

def load_vector_store(persist_path):
    """Carga la base de datos vectorial desde disco."""
    try:
        logging.info(f"🧠 Cargando base vectorial desde: {persist_path}")
        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
        vectordb = Chroma(persist_directory=persist_path, embedding_function=embeddings)
        logging.info("✅ Base vectorial cargada correctamente.")
        return vectordb
    except Exception as e:
        logging.error(f"❌ Error cargando la base vectorial: {e}")
        traceback.print_exc()
        return None

def query_llm(query, vectordb):
    """Consulta el modelo LLM local usando el contexto de los documentos más relevantes."""
    try:
        # Realiza la búsqueda en la base de datos vectorial
        logging.info("🔍 Buscando en la base vectorial...")
        results = vectordb.similarity_search(query, k=3)  # Tomar los 3 resultados más relevantes
        if not results:
            logging.warning("⚠️ No se encontraron resultados relevantes.")
            return "Lo siento, no pude encontrar información relevante."

        # Concatenar los resultados más relevantes
        context = "\n".join([result.page_content for result in results])
        logging.info("✅ Resultados encontrados, creando prompt para LLM...")

        # Configuración de Ollama (modelo local)
        llm = OllamaLLM(model="llama3")  # Puedes cambiarlo por otro modelo como llama3

        # Pregunta al modelo con el contexto
        prompt = f"Contexto:\n{context}\n\nPregunta: {query}\nRespuesta:"
        response = llm(prompt)
        
        return response
    except Exception as e:
        logging.error(f"❌ Error en la consulta LLM: {e}")
        traceback.print_exc()
        return "Hubo un error procesando tu consulta."




In [ ]:
if __name__ == "__main__":
    logging.info("🚀 Iniciando consulta...")

    # Cargar base vectorial
    vectordb = load_vector_store(CHROMA_PATH)
    if not vectordb:
        logging.warning("⚠️ No se pudo cargar la base vectorial. Finalizando.")
        exit()

In [ ]:
# Ingresar consulta
query = input("Escribe tu consulta: ")
if not query.strip():
    logging.warning("⚠️ No se ingresó ninguna consulta.")
    exit()

# Consultar al LLM
response = query_llm(query, vectordb)
logging.info(f"Pregunta al modelo:\n{query}")
logging.info(f"Respuesta del modelo:\n{response}")